<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/decisiontree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: Decision Tree #



##Inicialización##

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
)

In [3]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 74 (delta 35), reused 36 (delta 14), pack-reused 0
Unpacking objects: 100% (74/74), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


##Pre procesamiento##

In [5]:
pd.options.display.max_columns = None
df = pp.get_dataset()
df = pp.decisiontree_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [6]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth': np.arange(1, 32),
    'min_samples_leaf': np.arange(1, 32),
    "criterion": ["gini", "entropy"],
}

clf = DecisionTreeClassifier(random_state=42)

rgscv = RandomizedSearchCV(
    clf, params, n_iter=60, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [7]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8205882352941177
Best params {'min_samples_leaf': 26, 'max_depth': 28, 'criterion': 'entropy'}


##Evaluación del modelo##

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83        70
           1       0.84      0.61      0.70        51

    accuracy                           0.79       121
   macro avg       0.80      0.76      0.77       121
weighted avg       0.79      0.79      0.78       121



##Predicción##

In [10]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X_holdout = pp.decisiontree_preprocessing(X_holdout)
df2 = pd.DataFrame(rgscv.predict(X_holdout),columns=['volveria'])
df3 = pd.concat([X_holdout.id_usuario,df2],axis=1)
df3.to_csv('decisiontree.csv')
df3

,id_usuario,volveria
0,650,0
1,234,0
2,31,0
3,552,1
4,411,1
...,...,...
85,354,1
86,82,1
87,172,1
88,8,1
